# **VIRTUAL RACE ENGINEER**

In [2]:
import fastf1 as ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1.plotting
fastf1.plotting.setup_mpl()
import fastf1.api as fap
fastf1.Cache.enable_cache(r"D:\Prabhu\SEM 7\F1 Data")
import re

d:\Prabhu\SEM 7\F1 Data\.venv\Lib\site-packages\fastf1\api.py:32: UserWarning: `fastf1.api` will be considered private in future releases and potentially be removed or changed!
  warnings.warn("`fastf1.api` will be considered private in future releases and "


### LOADING SESSION

In [3]:
session = ff.get_session(2023,'Dutch','R')
session.load()

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22', '20', '63', '24', '16', '2']


In [4]:
laps=session.laps
drivers=laps['Driver'].unique()

### FETCHING TELEMETRY DATA

In [44]:
total_telemetry=[]
for i in drivers:
    drivers_laps=laps.pick_driver(i)
    driver_telemetry=drivers_laps.get_car_data()

    driver_telemetry['Driver']=i
    total_telemetry.append(driver_telemetry)
telemetry=pd.concat(total_telemetry,ignore_index=True)


### FETCHING WEATHER,TYRE,LAP AND POSITION DATA 

In [45]:
weather_data=pd.DataFrame(fap.weather_data(session.api_path))
tyre_data = laps[['Driver', 'LapNumber', 'Compound', 'FreshTyre','TyreLife']]
lap_data=laps[['Driver','LapNumber','LapTime','Sector1Time','Sector2Time','Sector3Time','IsAccurate']]
position_data=laps[['Driver','LapNumber','Position']]

req            INFO 	Using cached data for weather_data


### FETCHING RACE CONTROL MESSAGES

In [41]:
rcm_data = pd.DataFrame(fap.race_control_messages(session.api_path))
msg=rcm_data[['Time','Status','Message']]

#Filter for Yellow, Red, VSC, SC 
yellow_flags_key=['YELLOW','DOUBLE YELLOW']
red_flags_key=['RED']
vsc_key=['VSC','VIRTUAL SAFETY CAR']
sc_key=['SC','SAFETY CAR']

#Searching using the keyword in REGEX
yellow_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in yellow_flags_key))]
red_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in red_flags_key))]
vsc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in vsc_key))]
sc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in sc_key))]


req            INFO 	Using cached data for race_control_messages
